<a href="https://colab.research.google.com/github/4k5h1t/Network-Anomaly-Detection/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn import metrics
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers

import tensorflow.python.keras as keras

from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Dense

from sklearn.metrics import classification_report

RANDOM_STATE_SEED = 12

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Kaggle

Mounted at /content/gdrive
/content/gdrive/My Drive/Kaggle


In [ ]:
df = pd.read_csv('02-14-2018.csv')
df = df.drop(['Timestamp'], axis=1)
df = df[df.Label == 'Benign']
df = df.drop(['Label'], axis=1)

df = df.fillna(value=0)
df = df.replace([np.inf, -np.inf], 0)
df = df.clip(lower=0, upper=1)

df.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,0,1,1,0,0,0,0,0,0.0,...,0,0,0.0,0.0,0,0,1.0,1.0,1,1
1,0,0,1,1,0,0,0,0,0,0.0,...,0,0,0.0,0.0,0,0,1.0,1.0,1,1
2,0,0,1,1,0,0,0,0,0,0.0,...,0,0,0.0,0.0,0,0,1.0,1.0,1,1
3,1,1,1,1,1,1,1,1,0,1.0,...,1,1,0.0,0.0,0,0,0.0,0.0,0,0
4,1,1,1,1,1,1,1,1,0,1.0,...,1,1,0.0,0.0,0,0,0.0,0.0,0,0


In [ ]:
int_columns = df.select_dtypes(include=['int'])
int_columns = int_columns.astype(float)
df[int_columns.columns] = int_columns
train, test = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667626 entries, 0 to 1048574
Data columns (total 78 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           667626 non-null  int64  
 1   Protocol           667626 non-null  int64  
 2   Flow Duration      667626 non-null  int64  
 3   Tot Fwd Pkts       667626 non-null  int64  
 4   Tot Bwd Pkts       667626 non-null  int64  
 5   TotLen Fwd Pkts    667626 non-null  int64  
 6   TotLen Bwd Pkts    667626 non-null  int64  
 7   Fwd Pkt Len Max    667626 non-null  int64  
 8   Fwd Pkt Len Min    667626 non-null  int64  
 9   Fwd Pkt Len Mean   667626 non-null  float64
 10  Fwd Pkt Len Std    667626 non-null  float64
 11  Bwd Pkt Len Max    667626 non-null  int64  
 12  Bwd Pkt Len Min    667626 non-null  int64  
 13  Bwd Pkt Len Mean   667626 non-null  float64
 14  Bwd Pkt Len Std    667626 non-null  float64
 15  Flow Byts/s        667626 non-null  float64
 16  F

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(78,)),
       
    layers.Dense(78, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(50, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(36, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(20, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(36, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(50, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(78, activation='relu')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 78)                6162      
_________________________________________________________________
dropout (Dropout)            (None, 78)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                5056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3250      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 36)                1

In [ ]:
model.compile(loss='mean_squared_error', optimizer = 'adam')
model.fit(train, train, epochs=3, verbose=1)

Epoch 1/3
15648/15648 [==============================] - 78s 5ms/step - loss: 0.0374
Epoch 2/3
15648/15648 [==============================] - 66s 4ms/step - loss: 0.0262
Epoch 3/3
15648/15648 [==============================] - 64s 4ms/step - loss: 0.0258


In [ ]:
pred = model.predict(test)
score1 = np.sqrt(metrics.mean_squared_error(pred, test))

In [ ]:
score1

0.2944753847054788